In [0]:
!pip install torch torchvision
!pip install emoji

    100% |████████████████████████████████| 591.8MB 30kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61eec000 @  0x7f1c6e2312a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 14.2MB/s 
    100% |████████████████████████████████| 2.0MB 6.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
!ls

glove.6B.50d.txt  glove.6B.zip	sample_data  tesss.csv	train_emoji.csv


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm *.*.*00d.txt

--2019-01-07 14:23:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-01-07 14:23:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  20.9MB/s    in 87s     

2019-01-07 14:25:17 (9.44 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
%matplotlib inline

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import csv
import emoji
import matplotlib.pyplot as plt
from torch.autograd import Variable

torch.__version__

'1.0.0'

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [0]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

In [0]:
def read_csv(filename):
    phrase = []
    emoji = []

    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0].strip())
            emoji.append(row[1].strip())

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

In [0]:
from google.colab import files
uploaded = files.upload()

Saving tesss.csv to tesss.csv
Saving train_emoji.csv to train_emoji.csv


Load the dataset using the code below. We split the dataset between training (127 examples) and testing (56 examples).

In [0]:
X_train, Y_train = read_csv('train_emoji.csv')
X_test, Y_test = read_csv('tesss.csv')

In [0]:
len(X_train), len(X_test)

(132, 56)

In [0]:
# for i in range(len(X_train)):
#     print(i, X_train[i], label_to_emoji(Y_train[i]))

In [0]:
maxLen = len(max(X_train, key=len).split())
maxLen

10

In [0]:
index = 1
print(X_train[index], label_to_emoji(Y_train[index]))

I am proud of your achievements 😄


In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

- `word_to_index`: dictionary mapping from words to their indices in the vocabulary (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

In [0]:
# as a example
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


Convert all your training sentences into lists of indices, and then zero-pad all these lists so that their length is the length of the longest sentence.

In [0]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`. 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        
        # Initialize j to -1
        j = 0
        
        # Loop over the words of sentence_words
        for w in reversed(sentence_words):
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j-1] = word_to_index[w]
            # decrement j to j - 1
            j = j - 1
#         for w in sentence_words:
#             # Set the (i,j)th entry of X_indices to the index of the correct word.
#             X_indices[i, j] = word_to_index[w]
#             # decrement j to j + 1
#             j = j + 1
    
    return X_indices

In [0]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("\nX1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']

X1_indices = [[     0.      0.      0. 155345. 225122.]
 [     0.      0. 220930. 286375.  69714.]
 [151204. 192973. 302254. 151349. 394475.]]


In [0]:
X_train.shape, X_test.shape

((132,), (56,))

In [0]:
X_train = sentences_to_indices(X_train, word_to_index, maxLen)
X_test = sentences_to_indices(X_test, word_to_index, maxLen)

In [0]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(Y_test))

# dataloaders
batch_size = 4

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
x, y = next(iter(train_loader))
x.shape

torch.Size([4, 10])

In [0]:
# for i in range(x.size(0)):
#     print(' '.join([index_to_word[int(xx)] for xx in x[i] if xx != 0]), label_to_emoji(int(y[i])))

In [0]:
# First checking if GPU is available

train_on_gpu = torch.cuda.is_available()

print('Training on GPU.') if train_on_gpu else print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]    
    
    weight = torch.FloatTensor(emb_matrix)
    embedding_layer = nn.Embedding.from_pretrained(weight.cuda())
    embedding_layer.weight.requires_grad=False
    return embedding_layer

In [0]:
class Emojify(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, output_size, hidden_dim, n_layers, word_to_vec_map, word_to_index, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(Emojify, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers

        self.embedding = pretrained_embedding_layer(word_to_vec_map, word_to_index)
        embedding_dim = self.embedding.embedding_dim
    
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        pretrained_weight = np.array(pretrained_weight)
    
    
    
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        
        # linear and logsoftmax layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        x = x.type(torch.cuda.LongTensor)

        # embeddings and lstm_out
        
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # logsoftmax function
        logsoftmax_out = self.logsoftmax(out)
        
        # reshape to be batch_size first
        logsoftmax_out = logsoftmax_out.view(batch_size, -1)
        # return last logsoftmax output and hidden state
        return logsoftmax_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda()),
                  Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda()))
        
        return hidden

In [0]:
emb_matrix = pretrained_embedding_layer(word_to_vec_map, word_to_index)
vocab_len  = len(word_to_index) + 1

In [0]:
output_size = 5
hidden_dim = 128
n_layers = 2

net = Emojify(output_size, hidden_dim, n_layers, word_to_vec_map, word_to_index,drop_prob=0.5)

print(net)

NameError: ignored

In [0]:
# for param in model.parameters():
#     param.requires_grad = False
#     break
    
criterion =  nn.NLLLoss()   
optimizer1 = torch.optim.Adam(net.lstm.parameters(),lr=0.001)
optimizer2 = torch.optim.Adam(net.fc.parameters(),lr=0.001)


In [0]:
lr=0.01

criterion =  nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, net.parameters()), lr=lr)

In [0]:

# %%time

epochs = 50
batch_size=4
counter = 0
print_every = 50
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        
        output, h = net(inputs, h)
        
        # calculate the loss and perform backprop
        labels = labels.type(torch.cuda.LongTensor)
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#         nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer1.step()
        optimizer2.step()
        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            
            net.eval()
            with torch.no_grad():
                val_h = net.init_hidden(batch_size)
                val_losses = []
                for inputs, labels in test_loader:

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()
                    labels = labels.type(torch.cuda.LongTensor)

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output.squeeze(), labels)

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 2/50... Step: 50... Loss: 1.079356... Val Loss: 1.531807
Epoch: 4/50... Step: 100... Loss: 1.278362... Val Loss: 1.534689
Epoch: 5/50... Step: 150... Loss: 1.307775... Val Loss: 1.558298
Epoch: 7/50... Step: 200... Loss: 1.417094... Val Loss: 1.539577
Epoch: 8/50... Step: 250... Loss: 1.478668... Val Loss: 1.545313
Epoch: 10/50... Step: 300... Loss: 1.258510... Val Loss: 1.544250
Epoch: 11/50... Step: 350... Loss: 1.374892... Val Loss: 1.531473
Epoch: 13/50... Step: 400... Loss: 1.512365... Val Loss: 1.541032
Epoch: 14/50... Step: 450... Loss: 1.374421... Val Loss: 1.540887
Epoch: 16/50... Step: 500... Loss: 1.505918... Val Loss: 1.540011
Epoch: 17/50... Step: 550... Loss: 1.191117... Val Loss: 1.535961
Epoch: 19/50... Step: 600... Loss: 1.813581... Val Loss: 1.537650
Epoch: 20/50... Step: 650... Loss: 1.392972... Val Loss: 1.548061
Epoch: 22/50... Step: 700... Loss: 1.407580... Val Loss: 1.530825
Epoch: 23/50... Step: 750... Loss: 1.237919... Val Loss: 1.533456
Epoch: 25/50... 

### Using Dense Layer Only

In [0]:
E = pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [0]:
model = nn.Sequential(nn.Linear(50, 5),
                     nn.LogSoftmax(dim=1))
model.cuda()

Sequential(
  (0): Linear(in_features=50, out_features=5, bias=True)
  (1): LogSoftmax()
)

In [0]:
lr=0.003

criterion = nn.NLLLoss() # nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
for z in range(1000):
    c = 10
    L = 0
    for i, j in train_loader:
        i = i.type(torch.cuda.LongTensor)
        j = j.type(torch.cuda.LongTensor)
        i = E(i)
        i = i.mean(dim=1)
        model.zero_grad()
        out = model(i)
        loss = criterion(out, j)
        L += loss.item()
        loss.backward()
        optimizer.step()
    if z%c==0:
        print(L/len(train_loader))

1.5956361004800508
1.1728925542397932
0.9668360605384364
0.8361904530814199
0.7458668730475686
0.6801947120464209
0.6299959719181061
0.5858768838824648
0.5492915736906456
0.5218693708831613
0.4958112673325972
0.47238741769935144
0.45135725086385553
0.4331042125369563
0.4184632924470035
0.40348580344156787
0.3893981580481385
0.37610306116667663
0.3628714658094175
0.3525622369665088
0.34128371558406134
0.33214528587731446
0.32220733797911444
0.3132116568811012
0.30486777063572046
0.30037839801022503
0.29106539713613916
0.2838682220740752
0.2772464968941428
0.27000878977053094
0.2637331052259965
0.261217376499465
0.254329598311222
0.24677884578704834
0.24318666394912836
0.2360672688845432
0.23111939204461646
0.22677865985668066
0.22521896886103082
0.21899127689274875
0.2143439081582156
0.21106533977118405
0.20867258310317993
0.20282769112875967
0.2006824237830711
0.19736430410182837
0.19238117156606732
0.1902822716669603
0.1865366299947103
0.18525923066066974
0.18361716288508792
0.1778845